<a href="https://colab.research.google.com/github/majo581/TALLERES/blob/main/2_taller_regresion_polinomica_Esquivel_Gonzalez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/2_Taller_Regresion_Polinomica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Regresión Polinómica, Subajuste, Sobreajuste y Regularización**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

-Yuli Esquivel
-Maria Jose Gonzalez

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Reg_Polin_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

30 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

## **Situación**

Una importante firma de inversión inmobiliaria te ha contratado como consultor de ciencia de datos. Su proceso actual de valoración de propiedades es lento y se basa en la intuición de unos pocos expertos. Quieren que desarrolles un modelo de machine learning para predecir el precio de venta de las viviendas (`SalePrice`) de forma más precisa y sistemática.

Te han entregado el dataset "Ames Housing", que contiene una gran cantidad de información sobre viviendas vendidas recientemente. Tu tarea es construir el mejor modelo posible, pero más importante aún, justificar por qué tu modelo es robusto y fiable, explicando cómo has manejado la complejidad y el riesgo de sobreajuste.

### **Ejercicio 1: Carga y Preparación Inicial de los Datos**

Primero, carguemos las librerías necesarias y el dataset. Nos enfocaremos en un subconjunto de las variables numéricas para mantener el taller manejable, pero el principio se aplica a todo el dataset.

In [ ]:
# Importa las librerías necesarias
### TU CÓDIGO AQUÍ ###
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Importación de herramientas de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

Mejorar visualización de dataframes y gráficos

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

Cargar el dataset

In [ ]:
# Usamos el dataset Ames Housing desde su fuente original (requiere internet)
url = 'http://jse.amstat.org/v19n3/decock/AmesHousing.txt'
df = pd.read_csv(url, sep='\t')
df.head()

In [ ]:
# Para este taller, solo usaremos algunas columnas clave para simplificar.
df_sub = df[['Overall Qual', 'Gr Liv Area', 'SalePrice']].copy()
df_sub.columns = ['OverallQual', 'GrLivArea', 'SalePrice'] # Renombrar para facilidad
df_sub.head()

**Explicación de las variables del dataset reducido**

1. SalePrice: Precio de Venta.

Esta es la variable objetivo. Es el precio final por el cual se vendió la propiedad, medido en dólares estadounidenses.

2. OverallQual: Calidad General.

Es una variable ordinal que califica la calidad general del material y el acabado de la casa. Es una de las variables predictoras (X) más importantes.

Escala: Va de 1 a 10.

10: Muy Excelente

9: Excelente

...

2: Pobre

1: Muy Pobre

3. GrLivArea: Área Habitable sobre el Nivel del Suelo.

Es una variable numérica que mide el total de metros cuadrados de área habitable que está por encima del nivel del suelo. No incluye el área del sótano. Es una de las variables predictoras (X) más fuertes, ya que, lógicamente, casas más grandes tienden a ser más caras.

In [ ]:
df_sub.info()

### **Ejercicio 2: Dividir el conjunto de datos**

El método `.info()` muestra que no hay nulos en nuestro subconjunto. ¡Perfecto! Ahora, definamos nuestras variables `X` (predictoras) e `y` (objetivo) y dividamos los datos.

In [ ]:
# Definir X e y
X = df_sub[['OverallQual', 'GrLivArea']]
y = df_sub['SalePrice']

In [ ]:
# Dividir en entrenamiento y prueba (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Tamaño del set de Entrenamiento: {X_train.shape[0]} observaciones")
print(f"Tamaño del set de Prueba: {X_test.shape[0]} observaciones")

Tamaño del set de Entrenamiento: 2051 observaciones
Tamaño del set de Prueba: 879 observaciones


### **Ejercicio 3: Modelo con Sobreajuste**

Vamos a crear un modelo muy complejo para ver qué tan mal puede generalizar.

**Crear un Modelo Polinómico de Grado 5**

Usa `Pipeline` para combinar `PolynomialFeatures` (grado 5), `StandardScaler` y `LinearRegression`. Entrénalo con los datos de entrenamiento.

In [ ]:
# Crear el pipeline para el modelo polinómico
### TU CÓDIGO AQUÍ ###
pipeline_poly = Pipeline([
    ("poly_features", PolynomialFeatures(degree=25, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

In [ ]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
pipeline_poly.fit(X_train, y_train)


Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=25, include_bias=False)),
                ('scaler', StandardScaler()), ('model', LinearRegression())])

In [ ]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
y_train_pred_poly = pipeline_poly.predict(X_train)
y_test_pred_poly = pipeline_poly.predict(X_test)

rmse_train_poly = np.sqrt(mean_squared_error(y_train, y_train_pred_poly))
rmse_test_poly = np.sqrt(mean_squared_error(y_test, y_test_pred_poly))

print(f"RMSE en Entrenamiento (Poli-25): ${rmse_train_poly:,.2f} ")
print(f"RMSE en Prueba (Poli-25): ${rmse_test_poly:,.2f} ")

RMSE en Entrenamiento (Poli-25): $31,572.24 
RMSE en Prueba (Poli-25): $132,569,064.50 


**Pregunta:** ¿Qué indica la diferencia que observas entre el error de entrenamiento y el de prueba? ¿Le recomendarías este modelo a la firma inmobiliaria? ¿Por qué?


No recomendaría este modelo a la inmobiliaria porque está sobreentrenado. Aprendió tan bien los datos de práctica que no puede generalizar: mientras que con las casas que ya conoce tiene un error de $31,572, con casas nuevas el error se dispara a más de $132 millones, haciendo sus predicciones completamente inútiles para tomar decisiones reales.

### **Ejercicio 4: Aplicar Regularización**

Ahora, vamos a "curar" el sobreajuste. Usaremos los mismos `PolynomialFeatures` de grado 5, pero cambiaremos el modelo de regresión.

**Implementar Regresión Ridge**

Copia el pipeline anterior, pero reemplaza `LinearRegression` con `Ridge(alpha=10)`. `alpha` es la fuerza de la penalización.

In [ ]:
# Crear el pipeline para Ridge
pipeline_ridge = Pipeline([
    ("poly_features", PolynomialFeatures(degree=25, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=10))
])

In [ ]:
# Entrenar el modelo
pipeline_ridge.fit(X_train, y_train)

Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=25, include_bias=False)),
                ('scaler', StandardScaler()), ('model', Ridge(alpha=10))])

In [ ]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
y_train_pred_ridge = pipeline_ridge.predict(X_train)
y_test_pred_ridge = pipeline_ridge.predict(X_test)

rmse_train_ridge = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))

print(f"RMSE en Entrenamiento (Ridge): ${rmse_train_ridge:,.2f}")
print(f"RMSE en Prueba (Ridge): ${rmse_test_ridge:,.2f}")

RMSE en Entrenamiento (Ridge): $33,597.93
RMSE en Prueba (Ridge): $36,788.78


**Interpreta los resultados:**


El modelo con regularización Ridge funciona mucho mejor. Los errores de entrenamiento ($33,597) y prueba ($36,788) ahora son muy similares y razonables.

Esto indica que el modelo ya no está sobreentrenado: aprendió los patrones reales del mercado inmobiliario en lugar de memorizar los datos, por lo que SÍ recomendaría este modelo a la inmobiliaria porque hace predicciones consistentes y confiables tanto con casas conocidas como nuevas.

**Implementar Regresión Lasso**

Haz lo mismo, pero ahora con `Lasso(alpha=500, max_iter=10000)`. Lasso necesita un `alpha` más grande (porque la penalización L1 es diferente) y a veces más `max_iter` para converger.

In [ ]:
# Crear el pipeline para Lasso
pipeline_lasso = Pipeline([
    ("poly_features", PolynomialFeatures(degree=25, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=500, max_iter=10000))
])

In [ ]:
# Entrenar el modelo
pipeline_lasso.fit(X_train, y_train)

Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=25, include_bias=False)),
                ('scaler', StandardScaler()),
                ('model', Lasso(alpha=500, max_iter=10000))])

In [ ]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
y_train_pred_lasso = pipeline_lasso.predict(X_train)
y_test_pred_lasso = pipeline_lasso.predict(X_test)

rmse_train_lasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
rmse_test_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))

print(f"RMSE en Entrenamiento (Lasso): ${rmse_train_lasso:,.2f}")
print(f"RMSE en Prueba (Lasso): ${rmse_test_lasso:,.2f}")

RMSE en Entrenamiento (Lasso): $34,893.01
RMSE en Prueba (Lasso): $35,585.03


**Interpreta los resultados:**

El modelo Lasso solucionó el sobreajuste: los errores de entrenamiento ($34,893) y prueba ($35,585) son ahora muy similares y razonables, lo que indica que el modelo generaliza bien a datos nuevos y sus predicciones son confiables para la inmobiliaria.

**Selección de Variables con Lasso**

Una de las grandes ventajas de Lasso es que puede eliminar variables. Vamos a ver cuántas de las características polinómicas que creamos (ej. `OverallQual^2`, `GrLivArea^5`, `OverallQual * GrLivArea^4`, etc.) fueron eliminadas.

In [ ]:
# Extraer los coeficientes del modelo Lasso
lasso_coefficients = pipeline_lasso.named_steps['model'].coef_

In [ ]:
# Extraer los nombres de las características generadas
# Guíate por el siguiente código: feature_names = pipeline_lasso.named_steps['poly_features'].get_feature_names_out(X.columns)
feature_names = pipeline_lasso.named_steps['poly_features'].get_feature_names_out(X.columns)

In [ ]:
# Contar cuántos coeficientes son exactamente cero
zero_coefficients = np.sum(lasso_coefficients == 0)

In [ ]:
# Realizar un print de:
# Número total de características polinómicas generadas
# Número de características eliminadas por Lasso (coef = 0)
# Número de características CONSERVADAS
# Porcentaje de variables eliminadas
# Lista con los nombres de las características conservadas por Lasso (coef != 0)

total_features = len(feature_names)
conserved_features = total_features - zero_coefficients
percentage_eliminated = (zero_coefficients / total_features) * 100
conserved_feature_names = feature_names[lasso_coefficients != 0]

print(f"Número total de características polinómicas generadas: {total_features}")
print(f"Número de características eliminadas por Lasso (coef = 0): {zero_coefficients}")
print(f"Número de características CONSERVADAS: {conserved_features}")
print(f"Porcentaje de variables eliminadas: {percentage_eliminated:.2f}%")
print(f"Lista con los nombres de las características conservadas por Lasso (coef != 0):")
print(conserved_feature_names)

Número total de características polinómicas generadas: 350
Número de características eliminadas por Lasso (coef = 0): 332
Número de características CONSERVADAS: 18
Porcentaje de variables eliminadas: 94.86%
Lista con los nombres de las características conservadas por Lasso (coef != 0):
['OverallQual' 'OverallQual GrLivArea' 'OverallQual^2 GrLivArea'
 'OverallQual^3 GrLivArea' 'OverallQual^4 GrLivArea' 'OverallQual^7'
 'OverallQual^13 GrLivArea^7' 'OverallQual^12 GrLivArea^8'
 'OverallQual^14 GrLivArea^7' 'OverallQual^13 GrLivArea^8'
 'OverallQual^15 GrLivArea^7' 'OverallQual^14 GrLivArea^8'
 'OverallQual^16 GrLivArea^7' 'OverallQual^15 GrLivArea^8'
 'OverallQual^17 GrLivArea^7' 'OverallQual^16 GrLivArea^8'
 'OverallQual^25' 'OverallQual^18 GrLivArea^7']


**Interpreta los resultados:**

El modelo Lasso eliminó automáticamente el 94.8% de las variables, conservando solo 18 de las 350 características polinómicas originales. Esto demuestra su capacidad para seleccionar las características más importantes  principalmente combinaciones de "OverallQual" y "GrLivArea" descartando el ruido y simplificando el modelo mientras mantiene un buen rendimiento predictivo.

### **Ejercicio 5: Conclusión y Recomendación para el Cliente**

**Resumir los Resultados**

Crea un `DataFrame` de pandas que compare el RMSE de entrenamiento y prueba de los tres modelos (Polinómico, Ridge, Lasso) y ordena los modelos según el RMSE de prueba de menor a mayor.

In [ ]:
# Crear DataFrame de resultados
results_df = pd.DataFrame({
    'Modelo': ['Polinómico (Grado 25)', 'Ridge (Alpha=10)', 'Lasso (Alpha=500)'],
    'RMSE Entrenamiento': [rmse_train_poly, rmse_train_ridge, rmse_train_lasso],
    'RMSE Prueba': [rmse_test_poly, rmse_test_ridge, rmse_test_lasso]
})

# Ordenar por RMSE de Prueba
results_df = results_df.sort_values(by='RMSE Prueba')

# Mostrar el DataFrame
display(results_df)

,Modelo,RMSE Entrenamiento,RMSE Prueba
2,Lasso (Alpha=500),"34,893.01","35,585.03"
1,Ridge (Alpha=10),"33,597.93","36,788.78"
0,Polinómico (Grado 25),"31,572.24","132,569,064.50"


**Pregunta Final:**

Basado en tu análisis, ¿qué modelo le recomendarías a la firma inmobiliaria? Tu respuesta debe incluir:

1.  Una recomendación clara del mejor modelo.
Le recomiendo implementar el modelo Lasso (Alpha=500) ya que es el que mejor predice precios de casas nuevas, con el error más bajo ($35,585) y mayor confiabilidad.

2.  Una explicación en términos sencillos (para un gerente, no para un científico de datos) de por qué el modelo polinómico simple no era una buena idea, usando el concepto de "memorizar vs. generalizar".
El modelo polinómico simple funciona como un estudiante que "memoriza" las respuestas para el examen de práctica, pero cuando le toca un examen diferente (casas nuevas), reprueba porque no entendió los conceptos fundamentales. En cambio, los modelos con regularización "aprenden" las reglas reales del mercado, por eso predicen bien tanto con casas conocidas como nuevas.

3.  Una descripción de la ventaja principal del modelo Lasso en cuanto a la interpretabilidad y la selección de las características más importantes.
Lasso tiene la inteligencia de identificar automáticamente las características más importantes  en este caso, seleccionó solo 18 variables clave de las 350 originales, enfocándose principalmente en la calidad general de la casa y el área habitable. Esto no solo simplifica el modelo, sino que nos ayuda a entender qué factores realmente determinan el precio de una vivienda.